# Mount Google Drive to access files

---
# Imports & Config  

## Config

In [0]:
import os

base_dir = "/content/gdrive/My Drive/Masterarbeit"
object_detection_dir = os.path.join(base_dir, "ObjectDetection")
image_classification = os.path.join(base_dir, "ImageClassification")

classes_dict_file = object_detection_dir + "/classes.pkl"
objects_dict_file = object_detection_dir + "/objects.pkl"
images_dict_file = object_detection_dir + "/images.pkl"

## Imports

## Connect to GPU

In [0]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [2]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.87.93.34:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 6005017964602350324),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17537578373354958693),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 6340505640419897335),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13133146263969677974),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 9248057101367089695),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 281354276562644553),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13685777752437667117),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 4291063886531766030),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 2375977

## Mount GDrive

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Save & Load for pkls

In [0]:
import collections
import pickle as pkl

pkl_vars = {"objects": objects_dict_file,
            "images": images_dict_file,
            "classes": classes_dict_file}

def get_OD(var, force=False):
  if not var in globals() or force:
    print("loading {} from pickle".format(var))
    with open(pkl_vars[var], "rb") as f:
      return pkl.load(f)
    return
  elif var in globals():
    print("using already defined {}".format(var))
    return globals()[var]
  print("Failed to find something existing, creating new OrderedDictioniary for {}".format(var))
  return collections.OrderedDict()

def save_OD(var, local_var = None):
  if local_var is not None:
    print("Saving {} from handover variable".format(var))
    with open(pkl_vars[var], "+wb") as f:
      pkl.dump(local_var, f)
  elif var in globals():
    print("Saving {} from globals".format(var))
    with open(pkl_vars[var], "+wb") as f:
      pkl.dump(globals()[var], f)
  else:
    print("Failed to find something, unable to save {}".format(var))
    
def create_class_id(class_dict, prefix):
  class_int = int(next(reversed(class_dict))[2:])+1
  class_id = "{}{}".format(prefix, class_int)
  return class_id, class_int

## Replace print by prettyprint

In [0]:
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=2)
def print(kwargs):
  pp.pprint(kwargs)
  

---
# Object Detection

## Imports

In [0]:
!git clone https://github.com/tensorflow/models.git /content/tf
!cd /content/tf/research; protoc object_detection/protos/*.proto --python_out=.
!cd /content/tf/research; export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim; python /content/tf/research/object_detection/builders/model_builder_test.py

Cloning into '/content/tf'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 24141 (delta 24), reused 23 (delta 13), pack-reused 24097
Receiving objects: 100% (24141/24141), 563.36 MiB | 29.07 MiB/s, done.
Resolving deltas: 100% (14258/14258), done.
Checking out files: 100% (2767/2767), done.
......................
----------------------------------------------------------------------
Ran 22 tests in 0.124s

OK


In [0]:
import sys
import shutil
shutil.copytree("tf/research/object_detection" ,"object_detection")
shutil.copytree("tf/research/slim" ,"slim")

'slim'

In [0]:
import os
import sys

# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
#import matplotlib.pyplot as plt
#import tempfile
#from six.moves.urllib.request import urlopen
#from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
from datetime import datetime

#for xml parsing
from bs4 import BeautifulSoup

#...
#sys.path.append('tf/research')
#sys.path.append('tf/research/slim')
sys.path.append('slim')
from object_detection.utils import dataset_util, config_util
from object_detection import model_main

## Flags

In [0]:
class FLAGS_OD:
  image_dir = os.path.join(object_detection_dir, "Images")
  
  #tf records directory
  tf_records_dir = os.path.join(object_detection_dir, "TFRecords")
  #label_map directory
  label_map_dir = os.path.join(object_detection_dir, "LabelMaps")
  
  #class level to be considered
  class_level = [0,1]
  #how often the class of an object needs to occure to be considered
  min_class_occurence = 20
  
  #dataset split
  testing_percentage = 10
  validation_percentage = 10
  
  #pipeline config
  pipeline_config_dir = os.path.join(object_detection_dir, "ConfigFiles")
  
  #model checkpoints
  model_checkpoint_dir = os.path.join(object_detection_dir, "ModelCheckpoints")
  model_dir = os.path.join(object_detection_dir,"Models")

## Get Files


In [0]:
# From tensorflow/models/research/
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
!tar -xvf images.tar.gz
!tar -xvf annotations.tar.gz

In [0]:
from object_detection.dataset_tools.create_pet_tf_record import FLAGS, main

try:
  os.mkdir("data")
except:
  print("data dir already exists")

FLAGS.label_map_path = "object_detection/data/pet_label_map.pbtxt"
FLAGS.data_dir= ""
FLAGS.output_dir= "data"

tf.app.run(main)

In [0]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz
!tar -xvf ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03.tar.gz

In [0]:
model = "ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03"
import shutil
shutil.copy(os.path.join(FLAGS_OD.pipeline_config_dir,"{}.config".format(model)),"pipeline.config")
shutil.copy("object_detection/data/pet_label_map.pbtxt", "data/label_map.pbtxt")
if os.path.exists("model") and os.path.exists("{}".format(model)):
  shutil.rmtree("model")
  shutil.copytree("{}".format(model), "model")
elif os.path.exists("{}".format(model)):
  shutil.copytree("{}".format(model), "model")
else:
  print("couldn't find a model to copy") 

In [0]:
import os
import shutil
try:
  os.remove("annotations.tar.gz")
except:
  print("couldn't remove annotations.tar.gz")  
try:
  os.remove("images.tar.gz")
except:
  print("couldn't remove images.tar.gz") 
try:
  os.remove("{}.tar.gz".format(model))
except:
  print("couldn't remove {}.tar.gz".format(model)) 
try:
  shutil.rmtree("annotations")
except:
  print("couldn't remove annotations") 
try:
  shutil.rmtree("{}".format(model))
except:
  print("couldn't remove {}".format(model)) 
try:
  shutil.rmtree("images")
except:
  print("couldn't remove images") 

## Demo GCloud

In [0]:
!gcloud init

In [0]:
!gcloud config set project objectdetection-229309

Updated property [core/project].


In [0]:
!gsutil mb gs://holoselecta_objectdetection

Creating gs://holoselecta_objectdetection/...


In [0]:
!export PROJECT="objectdetection-229309"
!export YOUR_GCS_BUCKET="holoselecta_objectdetection"

In [0]:
!curl -H "Authorization: Bearer $(gcloud auth print-access-token)" https://ml.googleapis.com/v1/projects/objectdetection-229309:getConfig

{
  "serviceAccount": "service-142071610554@cloud-ml.google.com.iam.gserviceaccount.com",
  "serviceAccountProject": "727747150776",
  "config": {
    "tpuServiceAccount": "service-727747150776@cloud-tpu.iam.gserviceaccount.com"
  }
}


In [0]:
!export TPU_ACCOUNT=service-727747150776@cloud-tpu.iam.gserviceaccount.com

In [0]:
!gcloud projects add-iam-policy-binding objectdetection-229309  --member serviceAccount:service-727747150776@cloud-tpu.iam.gserviceaccount.com --role roles/ml.serviceAgent

Updated IAM policy for project [objectdetection-229309].
bindings:
- members:
  - serviceAccount:service-142071610554@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:142071610554-compute@developer.gserviceaccount.com
  - serviceAccount:142071610554@cloudservices.gserviceaccount.com
  role: roles/editor
- members:
  - serviceAccount:service-142071610554@cloud-ml.google.com.iam.gserviceaccount.com
  - serviceAccount:service-727747150776@cloud-tpu.iam.gserviceaccount.com
  role: roles/ml.serviceAgent
- members:
  - user:grundtob@gmail.com
  role: roles/owner
etag: BwV_9My7Jes=
version: 1


In [0]:
!gsutil -m cp data/pet_faces* gs://holoselecta_objectdetection/data/

Copying file://data/pet_faces_train.record-00000-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00001-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00005-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00004-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00002-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00003-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00007-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00009-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00008-of-00010 [Content-Type=application/octet-stream]...
Copying file://data/pet_faces_train.record-00006-of-00010 [Content-Type=application/octet-s

In [0]:
!gsutil cp object_detection/data/pet_label_map.pbtxt gs://holoselecta_objectdetection/data/pet_label_map.pbtxt

Copying file://object_detection/data/pet_label_map.pbtxt [Content-Type=application/octet-stream]...
/ [1 files][  1.5 KiB/  1.5 KiB]                                                
Operation completed over 1 objects/1.5 KiB.                                      


In [0]:
!gsutil cp model/model.ckpt.* gs://holoselecta_objectdetection/data/

Copying file://model/model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://model/model.ckpt.index [Content-Type=application/octet-stream]...
Copying file://model/model.ckpt.meta [Content-Type=application/octet-stream]...
\ [3 files][ 18.8 MiB/ 18.8 MiB]                                                
Operation completed over 3 objects/18.8 MiB.                                     


In [7]:
import os
import shutil
shutil.copy(os.path.join(FLAGS_OD.pipeline_config_dir,"ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.config"),"ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.config")
!gsutil cp ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.config gs://holoselecta_objectdetection/data/pipeline.config

Copying file://ssd_mobilenet_v1_0.75_depth_quantized_300x300_pets_sync.config [Content-Type=application/octet-stream]...
/ [1 files][  4.6 KiB/  4.6 KiB]                                                
Operation completed over 1 objects/4.6 KiB.                                      


In [0]:
!cd /content/tf/research; bash object_detection/dataset_tools/create_pycocotools_package.sh /tmp/pycocotools
!cd /content/tf/research; python setup.py sdist
!cd /content/tf/research; (cd slim && python setup.py sdist)

# Run

## GCloud ML Engine

In [12]:
!cd /content/tf/research; gcloud ml-engine jobs submit training `whoami`_object_detection_`date +%s` --job-dir=gs://holoselecta_objectdetection/train --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz --module-name object_detection.model_tpu_main --runtime-version 1.9 --scale-tier BASIC_TPU --region us-central1 -- --model_dir=gs://holoselecta_objectdetection/train --tpu_zone us-central1 --pipeline_config_path=gs://holoselecta_objectdetection/data/pipeline.config

Job [root_object_detection_1548067435] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe root_object_detection_1548067435

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs root_object_detection_1548067435
jobId: root_object_detection_1548067435
state: QUEUED


In [13]:
!cd /content/tf/research; gcloud ml-engine jobs submit training `whoami`_object_detection_eval_validation_`date +%s` --job-dir=gs://holoselecta_objectdetection/train --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/tmp/pycocotools/pycocotools-2.0.tar.gz --module-name object_detection.model_main --runtime-version 1.9 --scale-tier BASIC_GPU --region us-central1 -- --model_dir=gs://holoselecta_objectdetection/train --pipeline_config_path=gs://holoselecta_objectdetection/data/pipeline.config --checkpoint_dir=gs://holoselecta_objectdetection/train

Job [root_object_detection_eval_validation_1548067548] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe root_object_detection_eval_validation_1548067548

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs root_object_detection_eval_validation_1548067548
jobId: root_object_detection_eval_validation_1548067548
state: QUEUED


In [15]:
!gcloud ml-engine jobs describe root_object_detection_1548067435

createTime: '2019-01-21T10:44:03Z'
endTime: '2019-01-21T11:05:14Z'
etag: -ybdZUFMutY=
jobId: root_object_detection_1548067435
startTime: '2019-01-21T10:44:47Z'
state: SUCCEEDED
trainingInput:
  args:
  - --model_dir=gs://holoselecta_objectdetection/train
  - --tpu_zone
  - us-central1
  - --pipeline_config_path=gs://holoselecta_objectdetection/data/pipeline.config
  jobDir: gs://holoselecta_objectdetection/train
  packageUris:
  - gs://holoselecta_objectdetection/train/packages/d2a170285dd8617bc226dea3f11311326b04fc66d23948930fd424d95693510d/object_detection-0.1.tar.gz
  - gs://holoselecta_objectdetection/train/packages/d2a170285dd8617bc226dea3f11311326b04fc66d23948930fd424d95693510d/slim-0.1.tar.gz
  - gs://holoselecta_objectdetection/train/packages/d2a170285dd8617bc226dea3f11311326b04fc66d23948930fd424d95693510d/pycocotools-2.0.tar.gz
  pythonModule: object_detection.model_tpu_main
  region: us-central1
  runtimeVersion: '1.9'
  scaleTier: BASIC_TPU
trainingOutput:
  consumedMLUnits:

#Tensorboard

In [0]:
import os
os.mkdir("graph_tmp")

In [22]:
!gsutil cp -r gs://holoselecta_objectdetection/train graph_tmp/

Copying gs://holoselecta_objectdetection/train/checkpoint...
Copying gs://holoselecta_objectdetection/train/eval_validation_data/events.out.tfevents.1548067903.cmle-training-8023542701284428815...
/ [2 files][  8.5 MiB/  8.5 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://holoselecta_objectdetection/train/events.out.tfevents.1548067794.cmle-training-251881038172604876...
Copying gs://holoselecta_objectdetection/train/graph.pbtxt...
Copying gs://holoselecta_objectdetection/train/model.ckpt-0.data-00000-of-00001...
Copying gs://holoselecta_objectdetection/train/model.ckpt-0.index...
Copying gs://holoselecta_objectdetection/train/model.ckpt-0.meta...
Copying gs://holoselecta_objectdetection/train/model.ckpt-2000.data-00000-

In [19]:
from tensorboardcolab import TensorBoardColab

tbc = TensorBoardColab(graph_path="graph_tmp")

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://62a3c04a.ngrok.io


# Docs

You can freeze variables by adding freeze_variables: option under "**train_config**" in pipeline config file. For example, **freeze_variables: ".*FeatureExtractor.*"** will freeze the feature extractor. 

You can find out all variable names that can be frozen by adding print("var.op.name: " + var.op.name ) statement into the for loop after line 46 in object_detection/utils/variables_helper.py

To only load ghe weights of the feature vector layers you can change your "load_all_detection_checkpoint_vars" line in the config file by: **load_all_detection_checkpoint_vars: false**